In [108]:
import os
import numpy as np
# import re
import pickle
# import sys
import pandas as pd

# pd.set_option('display.float_format', lambda x: '%.6f' % x)

# import geopandas as gpd
# import pandas as pd
# from shapely.geometry import Point

# sys.path.insert(1, '../../../ProcessEvents')
# from ProcessEventsFunctions import *
# # from Convert_to_Profiles_Functions import *
# # from Get_Events_Functions import *

# Function to check for part1 in strings or lists of strings
def does_not_contain_part0(x):
    # Check if x is a string or a list of strings
    if isinstance(x, str):
        return 'part0' not in x
    elif isinstance(x, list):
        return any('part0' not in item for item in x)  # Ensure part1 is not in any item
    return True  # If it's neither, we keep it


def contains_part0(x):
    # Check if x is a string or a list of strings
    if isinstance(x, str):
        return 'part0' in x
    elif isinstance(x, list):
        return any('part0' in item for item in x)
    return False




In [31]:
home_dir = '/nfs/a319/gy17m2a/PhD/'
home_dir2 = '/nfs/a161/gy17m2a/PhD/'

## Check number of events when combined into the dictionary

### Read  the processed data into a dataframe

In [196]:
# events_props_dict_present = []
# ems_present = ['bc005', 'bc006', 'bc007', 'bc009', 'bc010', 'bc011', 'bc012', 'bc013', 'bc015', 'bc016', 'bc017', 'bc018']
# for em in ems_present:
#     with open(home_dir +  f"ProcessedData/AMAX_Events/UKCP18_30mins/Present/event_props_dict_{em}.pickle", 'rb') as handle:
#         events_props_dict = pickle.load(handle)    
#     events_props_dict_present = events_props_dict_present + events_props_dict
# ## Join into one dataframe    
# present_allevents = pd.DataFrame(events_props_dict_present)
# present_allevents['Climate'] = 'Present'    

events_props_dict_future = []
ems_future = ['bb195'] # ['bb192', 'bb195', 'bb198', 'bb208', 'bb225','bb222', 'bb201', 'bb204', 'bb216', 'bb219', 'bb211'] # , 
for em in ems_future:
    with open(home_dir +  f"ProcessedData/AMAX_Events/UKCP18_30mins/Future/event_props_dict_{em}_new.pickle", 'rb') as handle:
        events_props_dict = pickle.load(handle)    
    events_props_dict_future = events_props_dict_future + events_props_dict
## Join into one dataframe    
future_allevents = pd.DataFrame(events_props_dict_future)
future_allevents['Climate'] = 'Future'        

In [21]:
with open(home_dir +  f"ProcessedData/AMAX_Events/NIMROD_30mins/event_props_dict.pickle", 'rb') as handle:
    events_props_dict_nimrod = pickle.load(handle)    
    
## Join into one dataframe    
nimrod_allevents = pd.DataFrame(events_props_dict_nimrod)

### Total number of events
Different number of events each EM, because its all events, no duplicates, and number of events which are the AMAX for different durations will vary

In [33]:
for em in ems_future:
    # This is the TOTAL number of events
    print(em, len([d for d in events_props_dict_future if d.get('em') == em]))
    # This just does the same thing
    # this_em = future_allevents[future_allevents['em']==em]

bb198 92279


## Checking the number of part0 events
Get data for one em.  
Get data for one duration.  
Get data for one gauge, check if there are 19 events with part 0 in the string

Part0 rows sometimes has more than one entry per year, for the same duration. This is presumably because of CAM. However it's confusing because in that case I might have expected the filename to be Part1, but its not. 

WE CAN NO LONGER USE FILENAME TO SORT BECAUSE IT ONLY REFERS TO ONE OF THE EVENTS REPRESENTED BY EACH ROW

IF WANTED TO CHECK THIS ONCE AND FOR ALL, WOULD NEED TO IMPLEMENT SOMETHING IN PROCESSING FILE WHICH ASSIGNS THE FILENAME ROW WITH FLAGS FOR EACH DURATION AS TO WHETHER IT WAS A PART0 OR A PART1

In [197]:
durations =[0.5, 1,2,3,6,12,24]

In [218]:
## Get data for one ensemble member
for em in ['bb195']:
    print(em)
    this_em = future_allevents[future_allevents['em']==em].copy()
    
    ## Get data for one duration
    for duration in durations:
        print(f"duration : {duration}")
        
        #  Get events for just this duration
        future_allevents_thisdur = this_em[this_em['dur_for_which_this_is_amax'].apply(
            lambda x: isinstance(x, list) and str(duration) in x or x == str(duration))].copy()

        # Convert all durations and filenames to a list, even if just one value
        future_allevents_thisdur['filename'] = future_allevents_thisdur['filename'].apply(lambda x: x if isinstance(x, list) else [x])
        future_allevents_thisdur['dur_for_which_this_is_amax'] = future_allevents_thisdur['dur_for_which_this_is_amax'].apply(lambda x: x if isinstance(x, list) else [x])

        # Keep just the relevant filename for this duration
        future_allevents_thisdur['index_of_this_dur'] = future_allevents_thisdur['dur_for_which_this_is_amax'].apply(
            lambda x: x.index(str(duration)) if str(duration) in x else -1)
        # Create a new column to extract the corresponding filename using the index
        future_allevents_thisdur['corresponding_filename'] = future_allevents_thisdur.apply(
            lambda row: row['filename'][row['index_of_this_dur']] if row['index_of_this_dur'] != -1 and row['index_of_this_dur'] < len(row['filename']) else None, axis=1)

        ## Filter out the part1'ers
        future_allevents_thisdur = future_allevents_thisdur[~future_allevents_thisdur['corresponding_filename'].str.contains("part1", na=False)]
        print(f"{len(future_allevents_thisdur)} events for {duration}hrs")

        # Get data for one gauge, and find how many rows there are 
        # We would expect there to be 19
        for gauge_num in range(0,1294):
            if gauge_num not in [444,827,888]:
                this_gauge= future_allevents_thisdur[future_allevents_thisdur['gauge_num']==gauge_num].copy()

                if len(filtered_df) != 19:
                    print(gauge_num, len(filtered_df))
                    break


bb195
duration : 0.5
24529 events for 0.5hrs
duration : 1
24529 events for 1hrs
duration : 2
24529 events for 2hrs
duration : 3
24529 events for 3hrs
duration : 6
24529 events for 6hrs
duration : 12
24529 events for 12hrs
duration : 24
24529 events for 24hrs


In [65]:
# ## Get data for one duration
# for duration in durations:
#     nimrod_allevents_thisdur = nimrod_allevents[nimrod_allevents['dur_for_which_this_is_amax'].apply(
#         lambda x: isinstance(x, list) and str(duration) in x or x == str(duration))]
#     nimrod_allevents_thisdur.sort_values(by=['gauge_num', 'year'], ascending=[True, True])
#     print(f"{len(nimrod_allevents_thisdur)} events for {duration}hrs")

#     # Get data for one gauge, and find how many rows there are 
#     # We would expect there to be 19
#     for gauge_num in range(0,1294):
#         if gauge_num not in [444,827,888]:
#             this_gauge= nimrod_allevents_thisdur[nimrod_allevents_thisdur['gauge_num']==gauge_num]
# #             print(gauge_num, len(this_gauge))
#             if len(this_gauge) != 15:
#                 print(gauge_num, len(this_gauge))


In [66]:
# duration = '1'
# nimrod_allevents_thisdur = nimrod_allevents[nimrod_allevents['dur_for_which_this_is_amax'].apply(
#     lambda x: isinstance(x, list) and str(duration) in x or x == str(duration))]
# nimrod_allevents_thisdur.sort_values(by=['gauge_num', 'year'], ascending=[True, True])
# gauge_num= 0
# this_gauge= nimrod_allevents_thisdur[nimrod_allevents_thisdur['gauge_num']==gauge_num]
# this_gauge